In [ ]:
#Estre programa contém os código demonstrados na aula sobre Aplicação de ML+IoT para o Healthcare(previsão de arritmia)

In [2]:
#importando as bibliotecas
import pandas as pd #biblioteca utilizada para tratar os dados em formato de dataframe
import numpy as np # biblioteca utilizada para tratar vetores e matrizesimport matplotlib.pyplot as plt  #utilizapa para construir os gráficos em um formato similar ao "Matlab"
from sklearn.preprocessing import MinMaxScaler, LabelEncoder #utilizada para realizar o preprocessamento dos dados
from sklearn.model_selection import train_test_split #utilizada para realizar o divisão entre dados para treinamento e teste
from sklearn.metrics import confusion_matrix, accuracy_score #utilizada para verificar a acurácia do modelo construído
from sklearn.naive_bayes import GaussianNB # utilizada para construir o modelo de classificação naive_bayes
import seaborn as sns #utilizada para constuir os gráficos em uma forma mais "bonita"
import matplotlib.pyplot as plt #biblioteca para realizar a construção dos gráficos
from sklearn.svm import SVC #utilizada para importar o algoritmo SVM
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [3]:
#lendo o dataset no formato de dataframe atraves da funcao "read" do pandas
nomeArquivo = '/home/igorbaio/Documentos/BootcampAnalistaMachineLearning/Modulo_1/Desafio/comp_bikes_mod.csv'
dataset = pd.read_csv(nomeArquivo, sep=",")#realiza a leitura do banco de dados

In [4]:
#print do dataset
dataset.head() # são 76 colunas, mas nem todas serão utilizadas para realizar a previsão de doença cardíaca

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1.0,NaN,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,NaN,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.0,NaN,0.22,0.2727,0.80,0.0,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.80,0.0,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.75,0.0,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,NaN,1.0,0.24,0.2879,0.75,0.0,0.0,1.0,1.0


In [5]:
dataset.shape # mostra a dimensão do dataset

(17379, 17)

In [6]:
#conhecendo o dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     15641 non-null  float64
 1   dteday      15641 non-null  object 
 2   season      15641 non-null  float64
 3   yr          15641 non-null  float64
 4   mnth        15641 non-null  float64
 5   hr          15641 non-null  float64
 6   holiday     15641 non-null  float64
 7   weekday     15641 non-null  float64
 8   workingday  15641 non-null  float64
 9   weathersit  15641 non-null  float64
 10  temp        15641 non-null  float64
 11  atemp       15641 non-null  float64
 12  hum         15641 non-null  float64
 13  windspeed   15641 non-null  float64
 14  casual      15641 non-null  float64
 15  registered  15641 non-null  float64
 16  cnt         15641 non-null  float64
dtypes: float64(16), object(1)
memory usage: 2.3+ MB


In [7]:
dataset = dataset.dropna(axis=0,subset=['dteday'])

**Existem vários dados nulos**

In [8]:
#tratando os dados nulos
dataset.fillna(dataset.mean(), inplace=True) #substitui os dados que estão como NAN pela média dos valores na coluna
dataset.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.000000,1.422432,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.000000,1.000000,0.22,0.2727,0.80,0.0000,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.000000,1.000000,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.683053,1.000000,0.24,0.2879,0.75,0.0000,0.0,1.0,1.0
5,6.0,2011-01-01,1.0,0.0,1.0,5.0,0.0,6.0,0.000000,2.000000,0.24,0.2576,0.75,0.0896,0.0,1.0,1.0


**Preparando os dados**

In [9]:
dataset_to_array = np.array(dataset) #transforma o dataframe em array para facilitar a escolha dos dados a serem utilizados

In [10]:
previsores = dataset_to_array[:, 0:16]
previsores

array([[2.0, '2011-01-01', 1.0, ..., 0.0, 8.0, 32.0],
       [3.0, '2011-01-01', 1.0, ..., 0.0, 5.0, 27.0],
       [4.0, '2011-01-01', 1.0, ..., 0.0, 3.0, 10.0],
       ...,
       [17376.0, '2012-12-31', 1.0, ..., 0.1642, 35.416317248241064,
        153.280624556423],
       [8679.717496443813, '2012-12-31', 1.0, ..., 0.1343, 13.0, 48.0],
       [8679.717496443813, '2012-12-31', 2.5079297347272598, ..., 0.1343,
        12.0, 37.0]], dtype=object)

In [11]:
target = dataset_to_array[:,16] # esse é o vetor de saída (target)
target= target.astype('int') #indica o tipo de dados
#target[target>0] = 1 # 0 para o coração saudável e 1 para problema detectado
target

array([40, 32, 13, ..., 89, 61, 49])

In [12]:
dataset['dteday'] = pd.to_datetime(dataset.iloc[:, 1].values)

In [13]:
dataset_aux = dataset
dataset_aux.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.000000,1.422432,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.000000,1.000000,0.22,0.2727,0.80,0.0000,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.000000,1.000000,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.683053,1.000000,0.24,0.2879,0.75,0.0000,0.0,1.0,1.0
5,6.0,2011-01-01,1.0,0.0,1.0,5.0,0.0,6.0,0.000000,2.000000,0.24,0.2576,0.75,0.0896,0.0,1.0,1.0


In [14]:
dataset_to_train = dataset
dataset.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2.0,2011-01-01,1.0,0.0,1.0,1.0,0.0,6.0,0.000000,1.422432,0.22,0.2727,0.80,0.0000,8.0,32.0,40.0
2,3.0,2011-01-01,1.0,0.0,1.0,2.0,0.0,6.0,0.000000,1.000000,0.22,0.2727,0.80,0.0000,5.0,27.0,32.0
3,4.0,2011-01-01,1.0,0.0,1.0,3.0,0.0,6.0,0.000000,1.000000,0.24,0.2879,0.75,0.0000,3.0,10.0,13.0
4,5.0,2011-01-01,1.0,0.0,1.0,4.0,0.0,6.0,0.683053,1.000000,0.24,0.2879,0.75,0.0000,0.0,1.0,1.0
5,6.0,2011-01-01,1.0,0.0,1.0,5.0,0.0,6.0,0.000000,2.000000,0.24,0.2576,0.75,0.0896,0.0,1.0,1.0


**Iniciando a previsão**

In [15]:
dataset_to_train = np.column_stack((
    dataset_to_train.iloc[:, 2],
    dataset_to_train.iloc[:, 10],
    dataset_to_train.iloc[:, 11],
    dataset_to_train.iloc[:, 12],
    dataset_to_train.iloc[:, 13]
))
# Retirando algumas colunas 
#drop = ["instant", "dteday", "yr", "mnth", "hr", "holiday", "weekday","workingday","weathersit",
 #       "casual","registered","cnt"]
#dataset_to_train.drop(drop, inplace=True, axis=1)

#Salvando uma cópia do dataset para aplicar o hotencoder
#dataset_to_train2 = dataset_to_train.copy(deep=True)

In [ ]:
dataset_to_train

In [ ]:
dataset_to_train.shape

In [ ]:
#encontrando os dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_to_train, target, random_state = 2000)

In [ ]:
#cria o objeto SVM
modelSVM = SVC(kernel = 'linear') #escolha do kernel polinomial

In [ ]:
#aplica o treinamento ao modelo
modelSVM.fit(X_train, y_train)


**Analisando a performance do modelo**

In [ ]:
previsao = modelSVM.predict(X_test) #aplica o modelo para os dados de teste

In [ ]:
#encontra a acuracia do modelo de previsão utilizando o SVM 
accuracia = accuracy_score(y_test, previsao)
print ("Acuracia utilizando o SVM :" , accuracia , "\nEm porcentagem : ", round(accuracia*100) , "%\n")

In [ ]:
#criando a matriz de confusão
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, previsao) #gera a matriz de confusão
#df_cm = pd.DataFrame(cm, index = [i for i in "01234"],columns = [i for i in "01234"]) #cria o df com as classes
#plt.figure(figsize = (10,7)) #indica o tamanho da figura 
#sn.heatmap(cm, annot=True) #plota a figura
cm

In [ ]:
#realizando a análise de regressão
x=dataset["hum"].values  #variável independente 
Y=target #variável dependente 

In [ ]:
#importa o modelo de regressão linear univariada
from sklearn.linear_model import LinearRegression

In [ ]:
#Realiza a construção do modelo de regressão
reg= LinearRegression()
x_Reshaped=x.reshape((-1, 1)) #coloca os dados no formato 2D
regressao= reg.fit (x_Reshaped,Y) # encontra os coeficientes (realiza a regressão)

In [ ]:
#realiza a previsão
previsao=reg.predict(x_Reshaped)

In [ ]:
#análise do modelo
from sklearn.metrics import r2_score #método para o cálculo do R2 (coeficiente de determinação)

In [ ]:
#parâmetros encontrados
print('Y = {}X {}'.format(reg.coef_,reg.intercept_))

R_2 = r2_score(Y, previsao)  #realiza o cálculo do R2

print("Coeficiente de Determinação (R2):", R_2)


In [ ]:
#realizando a análise de regressão
x=dataset["casual"].values  #variável independente 
Y=dataset["cnt"].values #variável dependente 

In [ ]:
#importa o modelo de regressão linear univariada
from sklearn.linear_model import LinearRegression

In [ ]:
#Realiza a construção do modelo de regressão
reg= LinearRegression()
x_Reshaped=x.reshape((-1, 1)) #coloca os dados no formato 2D
regressao= reg.fit (x_Reshaped,Y) # encontra os coeficientes (realiza a regressão)

In [ ]:
#realiza a previsão
previsao=reg.predict(x_Reshaped)

In [ ]:
#análise do modelo
from sklearn.metrics import r2_score #método para o cálculo do R2 (coeficiente de determinação)

In [ ]:
#parâmetros encontrados
print('Y = {}X {}'.format(reg.coef_,reg.intercept_))

R_2 = r2_score(Y, previsao)  #realiza o cálculo do R2

print("Coeficiente de Determinação (R2):", R_2)


In [ ]:
#realizando a análise de regressão
x=dataset["casual"].values  #variável independente 
Y=dataset["cnt"].values #variável dependente 

In [ ]:
modelSVM = SVC(kernel = 'linear')


In [ ]:
from sklearn.tree import DecisionTreeRegressor
#Realiza a construção do modelo de regressão
regr_1 = DecisionTreeRegressor(max_depth=2)
x_Reshaped=x.reshape((-1, 1)) #coloca os dados no formato 2D
regressao= regr_1.fit(x_Reshaped,Y) # encontra os coeficientes (realiza a regressão)#realiza a previsão
previsao=regr_1.predict(x_Reshaped)

In [ ]:
#análise do modelo
from sklearn.metrics import r2_score #método para o cálculo do R2 (coeficiente de determinação)

In [ ]:
#parâmetros encontrados
print('Y = {}X {}'.format(reg.coef_,reg.intercept_))

R_2 = r2_score(Y, previsao)  #realiza o cálculo do R2

print("Coeficiente de Determinação (R2):", R_2)


In [ ]:
#realizando a análise de regressão
x=dataset["hum"].values  #variável independente 
Y=dataset["cnt"].values #variável dependente 

In [ ]:
modelSVM = SVC(kernel = 'linear')


In [ ]:
from sklearn.tree import DecisionTreeRegressor
#Realiza a construção do modelo de regressão
regr_1 = DecisionTreeRegressor(max_depth=2)
x_Reshaped=x.reshape((-1, 1)) #coloca os dados no formato 2D
regressao= regr_1.fit(x_Reshaped,Y) # encontra os coeficientes (realiza a regressão)#realiza a previsão
previsao=regr_1.predict(x_Reshaped)

In [ ]:
#análise do modelo
from sklearn.metrics import r2_score #método para o cálculo do R2 (coeficiente de determinação)

In [ ]:
#parâmetros encontrados
print('Y = {}X {}'.format(reg.coef_,reg.intercept_))

R_2 = r2_score(Y, previsao)  #realiza o cálculo do R2

print("Coeficiente de Determinação (R2):", R_2)


In [16]:
dataset.describe()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000,15641.000000
mean,8679.717496,2.507930,0.501563,6.542810,11.486779,0.027707,3.006535,0.683053,1.422432,0.496926,0.476055,0.627357,0.189552,35.416317,153.280625,189.165850
std,4755.683170,1.053079,0.474339,3.265233,6.558143,0.155709,1.900118,0.441739,0.606170,0.182997,0.163953,0.182921,0.116053,46.258180,143.619010,171.936005
min,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4816.000000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.360000,0.348500,0.490000,0.104500,5.000000,42.000000,50.000000
50%,8679.717496,2.507930,0.501563,6.542810,11.486779,0.000000,3.000000,1.000000,1.000000,0.496926,0.476055,0.627357,0.189552,21.000000,135.000000,168.000000
75%,12556.000000,3.000000,1.000000,9.000000,17.000000,0.000000,5.000000,1.000000,2.000000,0.640000,0.606100,0.770000,0.253700,43.000000,204.000000,259.000000
max,17376.000000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000
